# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv('../WeatherPy/data/city_weather.csv')
cities_df = pd.DataFrame(cities)
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Nizhnedevitsk,51.785170,38.442290,46.40,70,0,2.64,RU,1599943271
1,Tuktoyaktuk,88.244063,-140.906488,30.20,74,90,14.99,CA,1599943271
2,Hobart,-73.913309,139.835036,44.01,93,20,9.17,AU,1599943121
3,New Norfolk,-50.505946,136.849088,44.01,84,13,5.01,AU,1599943271
4,Punta Arenas,-71.501545,-104.907714,46.40,61,75,16.11,CL,1599943271
5,Severnyy,70.548989,65.661453,57.20,44,0,2.24,RU,1599943272
6,Puerto Ayora,-4.697495,-103.701758,75.20,60,20,19.46,EC,1599943273
7,Haimen,22.211955,117.567670,77.00,94,75,4.47,CN,1599943273
8,Vaini,-64.361985,-170.712351,69.80,73,20,8.05,TO,1599943274
9,Le Vauclin,15.957992,-58.586763,91.40,59,20,6.93,MQ,1599943274


In [3]:
# Copy the dataframe to begin cleaning the data
clean_city_data = cities_df

In [4]:
# Drop NA values inserted to handle missing values
clean_city_data = clean_city_data.dropna(inplace=True)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

FileNotFoundError: [Errno 2] File data/worldcities.csv does not exist: 'data/worldcities.csv'

In [5]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'hotel_df' is not defined

In [ ]:
# Add marker layer ontop of heat map


# Display figure
